# Hamilton-Jacobi-Bellman Solver for Optimal Insurance Control

This notebook demonstrates the use of the Hamilton-Jacobi-Bellman (HJB) solver for finding optimal insurance strategies through dynamic programming. The HJB approach provides theoretically optimal control policies by solving a partial differential equation that captures the full dynamics of the system.

## Key Advantages of HJB Approach:
1. **Global Optimality**: Finds globally optimal strategies, not just local optima
2. **State-Dependent Control**: Insurance parameters adapt to current financial state
3. **Forward-Looking**: Incorporates future uncertainty into current decisions
4. **Theoretical Guarantee**: Provides provably optimal strategies under model assumptions

Author: Alex Filiakov  
Date: 2025-01-26

In [ ]:
# Setup and imports
import sys
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# Configure plotting
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Import our modules
from ergodic_insurance.hjb_solver import (
    StateVariable, ControlVariable, StateSpace,
    LogUtility, PowerUtility, ExpectedWealth,
    HJBProblem, HJBSolver, HJBSolverConfig,
    create_custom_utility
)
from ergodic_insurance.optimal_control import (
    ControlSpace, StaticControl, HJBFeedbackControl,
    TimeVaryingControl, OptimalController,
    create_hjb_controller
)
from ergodic_insurance.config import ManufacturerConfig, GrowthConfig
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.simulation import Simulation

print("Modules loaded successfully!")

## Part 1: Simple 2D HJB Problem

We'll start with a simplified 2D problem (wealth and time) to demonstrate the core concepts.

In [ ]:
# Define state space (wealth and time)
state_variables = [
    StateVariable(
        name="wealth",
        min_value=1e6,
        max_value=1e8,
        num_points=30,
        log_scale=True  # Log scale for wealth
    ),
    StateVariable(
        name="time",
        min_value=0,
        max_value=5,
        num_points=20,
        log_scale=False
    )
]

state_space = StateSpace(state_variables)
print(f"State space dimensions: {state_space.ndim}")
print(f"State space shape: {state_space.shape}")
print(f"Total grid points: {state_space.size}")

In [ ]:
# Define control variables (insurance limit and retention)
control_variables = [
    ControlVariable(
        name="limit",
        min_value=1e6,
        max_value=3e7,
        num_points=15
    ),
    ControlVariable(
        name="retention",
        min_value=1e5,
        max_value=5e6,
        num_points=15
    )
]

print(f"Control dimensions: {len(control_variables)}")
print(f"Limit range: ${control_variables[0].min_value/1e6:.1f}M - ${control_variables[0].max_value/1e6:.1f}M")
print(f"Retention range: ${control_variables[1].min_value/1e6:.1f}M - ${control_variables[1].max_value/1e6:.1f}M")

## Part 2: Utility Functions

We'll demonstrate the three built-in utility functions and show how to create a custom one.

In [ ]:
# Compare different utility functions
wealth_range = np.linspace(1e6, 1e8, 100)

# Create utility functions
log_utility = LogUtility()
power_utility_2 = PowerUtility(risk_aversion=2.0)
power_utility_4 = PowerUtility(risk_aversion=4.0)
linear_utility = ExpectedWealth()

# Evaluate utilities
log_values = log_utility.evaluate(wealth_range)
power2_values = power_utility_2.evaluate(wealth_range)
power4_values = power_utility_4.evaluate(wealth_range)
linear_values = linear_utility.evaluate(wealth_range)

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Utility functions
ax = axes[0]
ax.plot(wealth_range/1e6, log_values - log_values[0], label='Log Utility', linewidth=2)
ax.plot(wealth_range/1e6, power2_values - power2_values[0], label='Power (γ=2)', linewidth=2)
ax.plot(wealth_range/1e6, power4_values - power4_values[0], label='Power (γ=4)', linewidth=2)
ax.plot(wealth_range/1e6, (linear_values - linear_values[0])/1e6, label='Linear (scaled)', linewidth=2)
ax.set_xlabel('Wealth ($M)')
ax.set_ylabel('Utility (normalized)')
ax.set_title('Utility Functions Comparison')
ax.legend()
ax.grid(True, alpha=0.3)

# Marginal utilities
ax = axes[1]
ax.loglog(wealth_range/1e6, log_utility.derivative(wealth_range), label='Log Utility', linewidth=2)
ax.loglog(wealth_range/1e6, power_utility_2.derivative(wealth_range), label='Power (γ=2)', linewidth=2)
ax.loglog(wealth_range/1e6, power_utility_4.derivative(wealth_range), label='Power (γ=4)', linewidth=2)
ax.set_xlabel('Wealth ($M)')
ax.set_ylabel('Marginal Utility')
ax.set_title('Marginal Utilities (log-log scale)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key Insights:")
print("- Log utility: Optimal for maximizing long-term growth rate (ergodic)")
print("- Power utility: Higher γ means more risk aversion")
print("- Linear utility: Risk-neutral, maximizes expected wealth")

## Part 3: Custom Utility Function

Here's how to create a custom utility function - we'll implement exponential utility.

In [ ]:
# Create custom exponential utility: U(w) = 1 - exp(-α*w)
alpha = 1e-7  # Risk aversion parameter

def exp_evaluate(w):
    """Exponential utility function."""
    return 1 - np.exp(-alpha * w)

def exp_derivative(w):
    """Marginal utility for exponential."""
    return alpha * np.exp(-alpha * w)

def exp_inverse_derivative(m):
    """Inverse of marginal utility."""
    return -np.log(m / alpha) / alpha

# Create custom utility
exponential_utility = create_custom_utility(
    exp_evaluate,
    exp_derivative,
    exp_inverse_derivative
)

# Test it
test_wealth = np.array([1e7, 5e7, 1e8])
exp_values = exponential_utility.evaluate(test_wealth)
exp_marginal = exponential_utility.derivative(test_wealth)

print("Custom Exponential Utility Test:")
for w, u, mu in zip(test_wealth/1e6, exp_values, exp_marginal):
    print(f"  Wealth: ${w:.0f}M → Utility: {u:.4f}, Marginal: {mu:.6f}")

# Verify inverse
recovered_wealth = exponential_utility.inverse_derivative(exp_marginal)
print(f"\nInverse derivative check: {np.allclose(recovered_wealth, test_wealth)}")

## Part 4: Solving the HJB Equation

Now we'll set up and solve a complete HJB problem for optimal insurance.

In [ ]:
# Define dynamics function
def company_dynamics(state, control, time):
    """Company wealth dynamics with insurance.
    
    state[..., 0] = wealth
    state[..., 1] = time
    control[..., 0] = insurance limit
    control[..., 1] = retention/deductible
    """
    wealth = state[..., 0]
    limit = control[..., 0]
    retention = control[..., 1]
    
    # Base growth rate
    growth_rate = 0.08  # 8% annual growth
    
    # Insurance premium (simplified model)
    # Higher limits cost more, lower retention costs more
    premium_rate = 0.015 * (limit / 2e7) * (1 + np.exp(-retention / 1e6))
    
    # Expected loss reduction from insurance
    loss_mitigation = 0.02 * (1 - np.exp(-limit / 1e7))
    
    # Net drift
    drift_wealth = wealth * (growth_rate - premium_rate + loss_mitigation)
    drift_time = np.ones_like(wealth)  # Time always advances at rate 1
    
    # Stack drifts
    drift = np.stack([drift_wealth, drift_time], axis=-1)
    return drift

# Define running cost (negative for maximization)
def running_reward(state, control, time):
    """Running reward function."""
    wealth = state[..., 0]
    
    # Use log utility for ergodic optimization
    utility = LogUtility()
    return utility.evaluate(wealth)

# Terminal value function
def terminal_value(state):
    """Terminal reward at end of time horizon."""
    wealth = state[..., 0]
    utility = LogUtility()
    return utility.evaluate(wealth)

print("Dynamics and reward functions defined")

In [ ]:
# Create and solve HJB problem
hjb_problem = HJBProblem(
    state_space=state_space,
    control_variables=control_variables,
    utility_function=LogUtility(),
    dynamics=company_dynamics,
    running_cost=running_reward,
    terminal_value=terminal_value,
    discount_rate=0.05,  # 5% discount rate
    time_horizon=5.0
)

# Solver configuration
solver_config = HJBSolverConfig(
    time_step=0.05,
    max_iterations=50,  # Limited for demonstration
    tolerance=1e-3,
    verbose=True
)

# Solve HJB equation
print("\nSolving HJB equation...")
print("="*50)

hjb_solver = HJBSolver(hjb_problem, solver_config)
value_function, optimal_policy = hjb_solver.solve()

print("\n" + "="*50)
print("HJB solution complete!")
print(f"Value function shape: {value_function.shape}")
print(f"Optimal policy keys: {list(optimal_policy.keys())}")

## Part 5: Visualizing the Solution

Let's visualize the value function and optimal policies.

In [ ]:
# Extract grids for plotting
wealth_grid = state_space.grids[0]
time_grid = state_space.grids[1]
W, T = np.meshgrid(wealth_grid, time_grid, indexing='ij')

# Create visualization
fig = plt.figure(figsize=(18, 12))

# Value function at t=0
ax1 = fig.add_subplot(2, 3, 1)
value_t0 = value_function[:, 0]
ax1.semilogx(wealth_grid/1e6, value_t0, 'b-', linewidth=2)
ax1.set_xlabel('Wealth ($M)')
ax1.set_ylabel('Value Function')
ax1.set_title('Value Function at t=0')
ax1.grid(True, alpha=0.3)

# Value function surface
ax2 = fig.add_subplot(2, 3, 2, projection='3d')
ax2.plot_surface(np.log10(W/1e6), T, value_function, cmap='viridis', alpha=0.8)
ax2.set_xlabel('Log Wealth ($M)')
ax2.set_ylabel('Time (years)')
ax2.set_zlabel('Value')
ax2.set_title('Value Function Surface')

# Optimal limit policy at t=0
ax3 = fig.add_subplot(2, 3, 3)
limit_t0 = optimal_policy['limit'][:, 0]
ax3.semilogx(wealth_grid/1e6, limit_t0/1e6, 'r-', linewidth=2)
ax3.set_xlabel('Wealth ($M)')
ax3.set_ylabel('Optimal Limit ($M)')
ax3.set_title('Optimal Insurance Limit at t=0')
ax3.grid(True, alpha=0.3)

# Optimal retention policy at t=0
ax4 = fig.add_subplot(2, 3, 4)
retention_t0 = optimal_policy['retention'][:, 0]
ax4.semilogx(wealth_grid/1e6, retention_t0/1e6, 'g-', linewidth=2)
ax4.set_xlabel('Wealth ($M)')
ax4.set_ylabel('Optimal Retention ($M)')
ax4.set_title('Optimal Retention at t=0')
ax4.grid(True, alpha=0.3)

# Policy surface for limit
ax5 = fig.add_subplot(2, 3, 5, projection='3d')
ax5.plot_surface(np.log10(W/1e6), T, optimal_policy['limit']/1e6, cmap='coolwarm', alpha=0.8)
ax5.set_xlabel('Log Wealth ($M)')
ax5.set_ylabel('Time (years)')
ax5.set_zlabel('Limit ($M)')
ax5.set_title('Optimal Limit Surface')

# Policy surface for retention
ax6 = fig.add_subplot(2, 3, 6, projection='3d')
ax6.plot_surface(np.log10(W/1e6), T, optimal_policy['retention']/1e6, cmap='coolwarm', alpha=0.8)
ax6.set_xlabel('Log Wealth ($M)')
ax6.set_ylabel('Time (years)')
ax6.set_zlabel('Retention ($M)')
ax6.set_title('Optimal Retention Surface')

plt.tight_layout()
plt.show()

print("Key Observations:")
print("1. Value function increases with wealth (diminishing returns due to log utility)")
print("2. Optimal insurance limit may vary with wealth level")
print("3. Retention level adapts to company's financial strength")
print("4. Policies change over time as we approach the terminal date")

## Part 6: Applying HJB Control in Simulation

Now let's use the HJB solution to control insurance in an actual simulation.

In [ ]:
# Create manufacturer
manufacturer_config = ManufacturerConfig(
    initial_assets=2e7,  # $20M starting assets
    asset_turnover_ratio=1.0,
    base_operating_margin=0.08,
    tax_rate=0.25,
    retention_ratio=0.6
)

# Create manufacturer instance
manufacturer = WidgetManufacturer(manufacturer_config)

print(f"Starting assets: ${manufacturer.assets/1e6:.1f}M")
print(f"Starting equity: ${manufacturer.equity/1e6:.1f}M")

In [ ]:
# Create control strategies for comparison

# 1. HJB Feedback Control
control_space = ControlSpace(
    limits=[(1e6, 3e7)],
    retentions=[(1e5, 5e6)]
)

hjb_strategy = HJBFeedbackControl(hjb_solver, control_space)
hjb_controller = OptimalController(hjb_strategy, control_space)

# 2. Static Control (baseline)
static_strategy = StaticControl(
    limits=[1.5e7],  # Fixed $15M limit
    retentions=[1e6]  # Fixed $1M retention
)
static_controller = OptimalController(static_strategy, control_space)

# 3. Time-Varying Control
time_varying_strategy = TimeVaryingControl(
    time_schedule=[0, 2.5, 5],
    limits_schedule=[[1e7], [2e7], [3e7]],  # Increasing limits
    retentions_schedule=[[5e5], [1e6], [2e6]]  # Increasing retentions
)
time_controller = OptimalController(time_varying_strategy, control_space)

print("Created 3 control strategies:")
print("1. HJB Feedback Control (optimal)")
print("2. Static Control (baseline)")
print("3. Time-Varying Control (predetermined schedule)")

In [ ]:
# Simulate with different controllers
simulation_years = 5
n_simulations = 100

def run_comparison_simulation(controller, manufacturer_cfg, years, n_sims):
    """Run multiple simulations with given controller."""
    results = []
    
    for sim in range(n_sims):
        # Reset manufacturer
        mfr = WidgetManufacturer(manufacturer_cfg)
        controller.reset()
        
        wealth_trajectory = [mfr.assets]
        
        for year in range(years):
            # Apply control
            insurance = controller.apply_control(mfr, time=year)
            
            # Simulate one year (simplified)
            # In production, would use full simulation engine
            revenue = mfr.assets * manufacturer_cfg.asset_turnover_ratio
            costs = revenue * (1 - manufacturer_cfg.base_operating_margin)
            
            # Simple loss model
            if np.random.random() < 0.1:  # 10% chance of loss
                loss = np.random.lognormal(14, 1)  # Log-normal losses
                covered_loss = min(loss, insurance.get_total_coverage())
                net_loss = loss - covered_loss
            else:
                net_loss = 0
            
            # Update wealth
            profit = revenue - costs - net_loss
            mfr.assets *= (1 + profit / mfr.assets)
            wealth_trajectory.append(mfr.assets)
        
        results.append({
            'final_wealth': mfr.assets,
            'trajectory': wealth_trajectory,
            'growth_rate': np.log(wealth_trajectory[-1] / wealth_trajectory[0]) / years
        })
    
    return pd.DataFrame(results)

print("Running simulations...")
print("This will take a moment...\n")

# Run simulations
hjb_results = run_comparison_simulation(hjb_controller, manufacturer_config, simulation_years, n_simulations)
static_results = run_comparison_simulation(static_controller, manufacturer_config, simulation_years, n_simulations)
time_results = run_comparison_simulation(time_controller, manufacturer_config, simulation_years, n_simulations)

print("Simulations complete!")

In [ ]:
# Compare results
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Final wealth distribution
ax = axes[0, 0]
ax.hist(hjb_results['final_wealth']/1e6, bins=20, alpha=0.5, label='HJB Control', color='blue')
ax.hist(static_results['final_wealth']/1e6, bins=20, alpha=0.5, label='Static Control', color='red')
ax.hist(time_results['final_wealth']/1e6, bins=20, alpha=0.5, label='Time-Varying', color='green')
ax.set_xlabel('Final Wealth ($M)')
ax.set_ylabel('Frequency')
ax.set_title('Final Wealth Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# Growth rate distribution
ax = axes[0, 1]
ax.hist(hjb_results['growth_rate']*100, bins=20, alpha=0.5, label='HJB Control', color='blue')
ax.hist(static_results['growth_rate']*100, bins=20, alpha=0.5, label='Static Control', color='red')
ax.hist(time_results['growth_rate']*100, bins=20, alpha=0.5, label='Time-Varying', color='green')
ax.set_xlabel('Annual Growth Rate (%)')
ax.set_ylabel('Frequency')
ax.set_title('Growth Rate Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# Sample trajectories
ax = axes[1, 0]
n_sample = 10
for i in range(n_sample):
    traj = hjb_results.iloc[i]['trajectory']
    ax.plot(range(len(traj)), np.array(traj)/1e6, 'b-', alpha=0.3, linewidth=0.5)
ax.set_xlabel('Year')
ax.set_ylabel('Wealth ($M)')
ax.set_title(f'Sample HJB Trajectories (n={n_sample})')
ax.grid(True, alpha=0.3)

# Performance comparison
ax = axes[1, 1]
metrics = pd.DataFrame({
    'Strategy': ['HJB\nControl', 'Static\nControl', 'Time-Varying\nControl'],
    'Mean Final Wealth': [
        hjb_results['final_wealth'].mean()/1e6,
        static_results['final_wealth'].mean()/1e6,
        time_results['final_wealth'].mean()/1e6
    ],
    'Median Growth Rate': [
        hjb_results['growth_rate'].median()*100,
        static_results['growth_rate'].median()*100,
        time_results['growth_rate'].median()*100
    ]
})

x = np.arange(len(metrics))
width = 0.35

bars1 = ax.bar(x - width/2, metrics['Mean Final Wealth'], width, label='Mean Final Wealth ($M)', color='steelblue')
ax2 = ax.twinx()
bars2 = ax2.bar(x + width/2, metrics['Median Growth Rate'], width, label='Median Growth Rate (%)', color='coral')

ax.set_xlabel('Strategy')
ax.set_ylabel('Mean Final Wealth ($M)', color='steelblue')
ax2.set_ylabel('Median Growth Rate (%)', color='coral')
ax.set_title('Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(metrics['Strategy'])
ax.tick_params(axis='y', labelcolor='steelblue')
ax2.tick_params(axis='y', labelcolor='coral')

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'${height:.1f}M', ha='center', va='bottom', fontsize=9)

for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}%', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Print summary statistics
print("\nPerformance Summary:")
print("="*60)
print(f"{'Strategy':<20} {'Mean Wealth':<15} {'Median Growth':<15} {'Std Dev':<15}")
print("-"*60)
print(f"{'HJB Control':<20} ${hjb_results['final_wealth'].mean()/1e6:.2f}M" 
      f"{' '*6}{hjb_results['growth_rate'].median()*100:.2f}%"
      f"{' '*8}${hjb_results['final_wealth'].std()/1e6:.2f}M")
print(f"{'Static Control':<20} ${static_results['final_wealth'].mean()/1e6:.2f}M"
      f"{' '*6}{static_results['growth_rate'].median()*100:.2f}%"
      f"{' '*8}${static_results['final_wealth'].std()/1e6:.2f}M")
print(f"{'Time-Varying':<20} ${time_results['final_wealth'].mean()/1e6:.2f}M"
      f"{' '*6}{time_results['growth_rate'].median()*100:.2f}%"
      f"{' '*8}${time_results['final_wealth'].std()/1e6:.2f}M")
print("="*60)

## Part 7: Convergence Analysis

Let's analyze the convergence of the HJB solver and solution quality.

In [ ]:
# Get convergence metrics
convergence_metrics = hjb_solver.compute_convergence_metrics()

print("HJB Solver Convergence Metrics:")
print("="*50)
print(f"Max HJB residual: {convergence_metrics['max_residual']:.6e}")
print(f"Mean HJB residual: {convergence_metrics['mean_residual']:.6e}")
print(f"Value function range: [{convergence_metrics['value_function_range'][0]:.4f}, "
      f"{convergence_metrics['value_function_range'][1]:.4f}]")

print("\nOptimal Policy Statistics:")
print("-"*50)
for control_name, stats in convergence_metrics['policy_stats'].items():
    print(f"{control_name}:")
    print(f"  Min: ${stats['min']/1e6:.2f}M")
    print(f"  Max: ${stats['max']/1e6:.2f}M")
    print(f"  Mean: ${stats['mean']/1e6:.2f}M")

## Part 8: Advanced - Higher Dimensional Problems

The HJB solver is designed to handle higher-dimensional problems. Here's an example with 3D state space.

In [ ]:
# Create 3D state space (wealth, time, loss history)
state_3d = StateSpace([
    StateVariable(
        name="wealth",
        min_value=1e6,
        max_value=1e8,
        num_points=10,  # Reduced for computational efficiency
        log_scale=True
    ),
    StateVariable(
        name="time",
        min_value=0,
        max_value=5,
        num_points=10
    ),
    StateVariable(
        name="loss_history",
        min_value=0,
        max_value=1e7,
        num_points=5  # Small for demonstration
    )
])

print(f"3D State Space:")
print(f"  Dimensions: {state_3d.ndim}")
print(f"  Shape: {state_3d.shape}")
print(f"  Total grid points: {state_3d.size:,}")
print(f"\nThis demonstrates scalability to higher dimensions.")
print(f"In practice, sparse grids or adaptive refinement would be used for efficiency.")

## Conclusions

This notebook demonstrated the Hamilton-Jacobi-Bellman solver for optimal insurance control:

### Key Takeaways:
1. **HJB provides globally optimal solutions** - Unlike local optimization methods, HJB finds the true optimum
2. **State-dependent control** - Insurance parameters adapt to current financial state
3. **Multiple utility functions** - Support for log (ergodic), power (CRRA), and custom utilities
4. **Scalable to higher dimensions** - Framework supports multi-dimensional state spaces
5. **Integration with simulation** - Seamless integration with existing simulation framework

### Performance Benefits:
- HJB control typically outperforms static and predetermined strategies
- Better risk-adjusted returns through dynamic adaptation
- Theoretical guarantees under model assumptions

### Next Steps:
1. **Performance optimization** - Add Numba JIT compilation for faster solving
2. **Sparse grids** - Implement sparse grid techniques for high-dimensional problems
3. **Model calibration** - Calibrate dynamics to historical data
4. **Robustness analysis** - Test sensitivity to model parameters
5. **Real-time control** - Implement online learning and adaptation

The HJB solver provides a powerful tool for optimal insurance decision-making,
offering theoretical rigor and practical applicability.